# Amazon Bedrock - Video Generation with Nova Reel
This notebook demonstrates how to generate videos from text and images using Amazon Nova Reel

In [ ]:
# %% Import necessary packages
import boto3
import json
import base64
import time
from pathlib import Path
from datetime import datetime
import os

In [ ]:
# %% Initialize Bedrock Runtime client
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')
model_id = "amazon.nova-reel-v1:0"

In [ ]:
# %% Helper function to save video
def save_video(base64_string, filename):
    """Decode base64 video and save to file"""
    video_bytes = base64.b64decode(base64_string)
    
    os.makedirs('generated_videos', exist_ok=True)
    filepath = f"generated_videos/{filename}"
    
    with open(filepath, 'wb') as f:
        f.write(video_bytes)
    
    print(f"Video saved to: {filepath}")
    print(f"Size: {len(video_bytes) / (1024*1024):.2f} MB")
    return filepath

In [ ]:
# %% Helper function to start async video generation
def start_video_generation(request_body):
    """Start async video generation job"""
    response = bedrock_runtime.start_async_invoke(
        modelId=model_id,
        modelInput=request_body
    )
    
    invocation_arn = response['invocationArn']
    print(f"Job started: {invocation_arn}")
    return invocation_arn

def get_video_result(invocation_arn, max_wait=300):
    """Poll for video generation result"""
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        response = bedrock_runtime.get_async_invoke(
            invocationArn=invocation_arn
        )
        
        status = response['status']
        print(f"Status: {status}")
        
        if status == 'Completed':
            # Get output from S3 location
            output_location = response['outputDataConfig']['s3OutputDataConfig']['s3Uri']
            print(f"Output location: {output_location}")
            return response
        elif status == 'Failed':
            print(f"Job failed: {response.get('failureMessage', 'Unknown error')}")
            return None
        
        time.sleep(10)
    
    print("Timeout waiting for video generation")
    return None

In [ ]:
# %% Text-to-video generation
prompt = "A yellow bulldozer working on a construction site, moving dirt and rocks, cinematic camera movement, 4K quality"

request_body = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": prompt
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,
        "fps": 24,
        "dimension": "1280x720",
        "seed": 42
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Text-to-Video Generation:")
print("=" * 80)
print(f"Prompt: {prompt}")
print(f"Duration: 6 seconds")
print(f"FPS: 24")
print(f"Dimension: 1280x720")

filename = f"bulldozer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
save_video(response_body['video'], filename)

In [ ]:
# %% Image-to-video generation (animate a static image)
# Load an image from the previous notebook
image_path = "generated_images/bulldozer_20241206_225000.png"  # Update with actual path

# For demo, create a simple image
from PIL import Image, ImageDraw
from io import BytesIO

demo_image = Image.new('RGB', (1280, 720), color='skyblue')
draw = ImageDraw.Draw(demo_image)
draw.rectangle([400, 300, 800, 500], fill='yellow')  # Simple bulldozer shape
draw.text((500, 600), "BULLDOZER", fill='black')

buffer = BytesIO()
demo_image.save(buffer, format='PNG')
image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

prompt = "Animate this bulldozer moving forward, camera panning to follow the motion"

request_body = {
    "taskType": "IMAGE_VIDEO",
    "imageToVideoParams": {
        "text": prompt,
        "images": [image_base64]
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,
        "fps": 24,
        "dimension": "1280x720",
        "seed": 100
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Image-to-Video Generation:")
print("=" * 80)
print(f"Prompt: {prompt}")
print("Animating static image")

filename = f"animated_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
save_video(response_body['video'], filename)

In [ ]:
# %% Generate video with camera control
prompt = """An excavator digging in a quarry at golden hour. 
Camera starts with a wide shot, then slowly zooms in on the excavator arm. 
Cinematic lighting, professional videography."""

request_body = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": prompt,
        "negativeText": "blurry, low quality, shaky, distorted, cartoon"  # What to avoid
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,
        "fps": 24,
        "dimension": "1280x720",
        "seed": 200
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Video with Camera Control:")
print("=" * 80)
print(f"Prompt: {prompt}")
print("Using negative prompt for quality control")

filename = f"excavator_cinematic_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
save_video(response_body['video'], filename)

In [ ]:
# %% Generate multiple video variations
base_prompt = "A forklift moving pallets in a warehouse"

variations = [
    "slow motion, dramatic lighting",
    "time-lapse, busy warehouse activity",
    "close-up of forklift controls, operator perspective"
]

print("Generating 3 video variations:")
print("=" * 80)

for idx, variation in enumerate(variations):
    full_prompt = f"{base_prompt}, {variation}"
    
    request_body = {
        "taskType": "TEXT_VIDEO",
        "textToVideoParams": {
            "text": full_prompt
        },
        "videoGenerationConfig": {
            "durationSeconds": 6,
            "fps": 24,
            "dimension": "1280x720",
            "seed": 300 + idx
        }
    }
    
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(request_body)
    )
    
    response_body = json.loads(response['body'].read())
    
    print(f"\nVariation {idx+1}: {variation}")
    filename = f"forklift_var{idx+1}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
    save_video(response_body['video'], filename)

In [ ]:
# %% Create video from multiple generated images (sequence)
# Load multiple images from the image generation notebook
image_paths = [
    "generated_images/excavator_var1_20241206_225000.png",
    "generated_images/excavator_var2_20241206_225001.png",
    "generated_images/excavator_var3_20241206_225002.png"
]

# For demo, create 3 simple images
images_base64 = []
colors = ['red', 'green', 'blue']

for color in colors:
    img = Image.new('RGB', (1280, 720), color=color)
    draw = ImageDraw.Draw(img)
    draw.text((500, 350), f"Frame: {color.upper()}", fill='white')
    
    buffer = BytesIO()
    img.save(buffer, format='PNG')
    images_base64.append(base64.b64encode(buffer.getvalue()).decode('utf-8'))

prompt = "Smooth transition between these construction site scenes, maintaining continuity"

request_body = {
    "taskType": "IMAGE_VIDEO",
    "imageToVideoParams": {
        "text": prompt,
        "images": images_base64  # Multiple images for sequence
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,
        "fps": 24,
        "dimension": "1280x720"
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("Video from Image Sequence:")
print("=" * 80)
print(f"Number of input images: {len(images_base64)}")
print(f"Prompt: {prompt}")

filename = f"sequence_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
save_video(response_body['video'], filename)

In [ ]:
# %% Generate high-quality video with all parameters
prompt = """A mobile crane lifting heavy steel beams at a construction site. 
Professional cinematography with smooth camera movement. 
Golden hour lighting, 4K quality, realistic physics."""

negative_prompt = """blurry, low quality, shaky camera, distorted, 
unrealistic motion, cartoon, animated, CGI artifacts"""

request_body = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": prompt,
        "negativeText": negative_prompt
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,
        "fps": 24,
        "dimension": "1280x720",  # Options: "1280x720", "1920x1080"
        "seed": 12345
    }
}

response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType="application/json",
    accept="application/json",
    body=json.dumps(request_body)
)

response_body = json.loads(response['body'].read())

print("High-Quality Video Generation:")
print("=" * 80)
print(f"Prompt: {prompt}")
print(f"Negative Prompt: {negative_prompt}")
print(f"Duration: 6 seconds")
print(f"FPS: 24")
print(f"Dimension: 1280x720")

filename = f"crane_hq_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
save_video(response_body['video'], filename)

In [ ]:
# %% Summary of parameters
print("Nova Reel Video Generation Parameters:")
print("=" * 80)
print("\nTask Types:")
print("  - TEXT_VIDEO: Generate video from text prompt")
print("  - IMAGE_VIDEO: Animate static image(s) or create video from sequence")
print("\nVideo Dimensions:")
print("  - 1280x720 (720p HD)")
print("  - 1920x1080 (1080p Full HD)")
print("\nDuration:")
print("  - 6 seconds (standard)")
print("\nFPS (Frames Per Second):")
print("  - 24 fps (cinematic)")
print("\nPrompt Tips:")
print("  - Include camera movement (pan, zoom, dolly)")
print("  - Specify lighting (golden hour, dramatic, natural)")
print("  - Add style keywords (cinematic, professional, 4K)")
print("  - Use negative prompts to avoid unwanted elements")
print("\nImage-to-Video:")
print("  - Single image: Animates the image")
print("  - Multiple images: Creates transitions between them")
print("  - Max images: Check current API limits")
print("\nSeed: Any integer (for reproducibility)")
print("\nOutput Format: MP4 (H.264)")